In [1]:
import json
import pandas as pd
import re
from google.cloud import bigquery
from google.cloud import bigquery_storage
from google.oauth2 import service_account
import os
from datetime import datetime

In [3]:
# Retrieve the output data from the JSON file
with open("mobile_de/output.json", mode="r", encoding='utf-8') as f:
    df_data_all_car_brands = json.load(f)
    df_data_all_car_brands = pd.DataFrame(df_data_all_car_brands)
    f.close()

# Drop the duplicates because the mileage filters applied above could have produced duplicates
df_data_all_car_brands = df_data_all_car_brands.drop_duplicates(["url_to_crawl"])

# Step 1: Clean the data
df_data_all_car_brands_cleaned = df_data_all_car_brands.copy()
df_data_all_car_brands_cleaned.replace(to_replace="", value=None, inplace=True)
df_data_all_car_brands_cleaned["leistung"] = df_data_all_car_brands_cleaned["leistung"].apply(lambda x: int(re.findall(pattern="(?<=\().*(?=\sPS)", string=x)[0].replace(".", "")) if x is not None else x)
df_data_all_car_brands_cleaned["preis"] = df_data_all_car_brands_cleaned["preis"].apply(lambda x: int(''.join(re.findall(pattern="\d+", string=x))) if x is not None else x)
df_data_all_car_brands_cleaned["kilometer"] = df_data_all_car_brands_cleaned["kilometer"].apply(lambda x: int(''.join(re.findall(pattern="\d+", string=x))) if x is not None else x)
df_data_all_car_brands_cleaned["fahrzeughalter"] = df_data_all_car_brands_cleaned["fahrzeughalter"].apply(lambda x: int(x) if x is not None else x)
df_data_all_car_brands_cleaned["standort"] = df_data_all_car_brands_cleaned["standort"].apply(lambda x: re.findall(pattern="[A-za-z]+(?=-)", string=x)[0] if x is not None else x)
df_data_all_car_brands_cleaned["crawled_timestamp"] = datetime.now()

In [5]:
# Step 2: Upload to bigquery
# First, set the credentials
key_path_home_dir = os.path.expanduser("~") + "/bq_credentials.json"
credentials = service_account.Credentials.from_service_account_file(
    key_path_home_dir, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

# Now, instantiate the client and upload the table to BigQuery
bq_client = bigquery.Client(project="web-scraping-371310", credentials=credentials)
bqstorageclient = bigquery_storage.BigQueryReadClient(credentials=credentials)

In [ ]:
job_config = bigquery.LoadJobConfig(
    schema = [
        bigquery.SchemaField("marke", "STRING"),
        bigquery.SchemaField("modell", "STRING"),
        bigquery.SchemaField("variante", "STRING"),
        bigquery.SchemaField("titel", "STRING"),
        bigquery.SchemaField("form", "STRING"),
        bigquery.SchemaField("fahrzeugzustand", "STRING"),
        bigquery.SchemaField("leistung", "FLOAT64"),
        bigquery.SchemaField("getriebe", "STRING"),
        bigquery.SchemaField("farbe", "STRING"),
        bigquery.SchemaField("preis", "INT64"),
        bigquery.SchemaField("kilometer", "FLOAT64"),
        bigquery.SchemaField("erstzulassung", "STRING"),
        bigquery.SchemaField("fahrzeughalter", "FLOAT64"),
        bigquery.SchemaField("standort", "STRING"),
        bigquery.SchemaField("fahrzeugbescheibung", "STRING"),
        bigquery.SchemaField("url_to_crawl", "STRING"),
        bigquery.SchemaField("page_rank", "INT64"),
        bigquery.SchemaField("total_num_pages", "INT64"),
        bigquery.SchemaField("crawled_timestamp", "TIMESTAMP"),
    ]
)
job_config.write_disposition = bigquery.WriteDisposition.WRITE_APPEND

# Upload the table
bq_client.load_table_from_dataframe(
    dataframe=df_data_all_car_brands_cleaned,
    destination="web-scraping-371310.crawled_datasets.lukas_mobile_de",
    job_config=job_config
).result()

## Data cleaning Porsche 992 GT3

In [8]:
query = """
    SELECT *
    FROM `web-scraping-371310.crawled_datasets.lukas_mobile_de`
    WHERE crawled_timestamp = (SELECT MAX(crawled_timestamp) FROM `web-scraping-371310.crawled_datasets.lukas_mobile_de`)
"""
df = pd.DataFrame(bq_client.query(query).to_dataframe(bqstorage_client=bqstorageclient, progress_bar_type="tqdm_notebook"))

Query is running:   0%|          |

Downloading:   0%|          |

In [9]:
print(f"The length of the dataset is: {len(df)}")

The length of the dataset is: 52035


#### Filter for Porsche 992

In [10]:
df_clean_1 = df[
    (df["marke"]=="Porsche") &
    (df["modell"]=="992")
].reset_index(drop=True)

In [11]:
df_clean_1

,marke,modell,variante,titel,form,fahrzeugzustand,leistung,getriebe,farbe,preis,kilometer,erstzulassung,fahrzeughalter,standort,fahrzeugbescheibung,url_to_crawl,page_rank,total_num_pages,crawled_timestamp
0,Porsche,992,None,Porsche 992,Sportwagen / Coupé,Beschädigt,503.0,Automatik,Schwarz,95000,116852.0,03/2016,NaN,DE,Verkaufe meinen Porsche 911 Carrera.\nIch habe...,https://suchen.mobile.de/fahrzeuge/details.htm...,1,25,2024-03-09 22:09:26.038992+00:00
1,Porsche,992,None,Porsche 992,Cabrio / Roadster,None,385.0,None,Schwarz Metallic,125000,33000.0,05/2020,2.0,DE,"Sehr gepflegter Garagen Wagen, keine Tiere, Ni...",https://suchen.mobile.de/fahrzeuge/details.htm...,9,25,2024-03-09 22:09:26.038992+00:00
2,Porsche,992,None,Porsche 992 Carrera,Sportwagen / Coupé,None,385.0,Automatik,Grau,129900,25938.0,01/2020,1.0,NL,Informationen auf Deutsch: \n \n Original nied...,https://suchen.mobile.de/fahrzeuge/details.htm...,11,25,2024-03-09 22:09:26.038992+00:00
3,Porsche,992,None,Porsche 992 Targa 4,Cabrio / Roadster,None,385.0,Automatik,Blau Metallic,149500,27400.0,11/2020,NaN,BE,Porsche 992 Targa 4 2020\nFirst registration 2...,https://suchen.mobile.de/fahrzeuge/details.htm...,15,25,2024-03-09 22:09:26.038992+00:00
4,Porsche,992,None,Porsche 992 Turbo S,Sportwagen / Coupé,None,650.0,Automatik,Silber Metallic,207894,55000.0,09/2020,2.0,NL,Informationen auf Deutsch: \n \n \n \nWeitere ...,https://suchen.mobile.de/fahrzeuge/details.htm...,23,25,2024-03-09 22:09:26.038992+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2222,Porsche,992,None,Porsche 992 911 992 Targa 4 GTS /PCCB/Matrix/A...,Cabrio / Roadster,None,480.0,Automatik,Schwarz Metallic,188900,2900.0,06/2022,1.0,DE,Sonderausstattung\n \nAblage-Paket\n \nAußen-/...,https://suchen.mobile.de/fahrzeuge/details.htm...,8,50,2024-03-09 22:09:26.038992+00:00
2223,Porsche,992,None,Porsche 992 911 GT3 RS / WEISSACH/LIFT/ PCCB/K...,Sportwagen / Coupé,Unfallfrei,525.0,Automatik,Silber Metallic,464990,370.0,04/2023,1.0,DE,"Wir haben, wonach du suchst!\nWir wählen unser...",https://suchen.mobile.de/fahrzeuge/details.htm...,6,50,2024-03-09 22:09:26.038992+00:00
2224,Porsche,992,None,Porsche 992 991/992 Carrera Cabriolet S*Sporta...,Cabrio / Roadster,None,450.0,Automatik,Grau Metallic,142980,694.0,10/2022,1.0,DE,"Wir freuen uns sehr, dass wir mit unserem Ange...",https://suchen.mobile.de/fahrzeuge/details.htm...,3,50,2024-03-09 22:09:26.038992+00:00
2225,Porsche,992,None,Porsche 992 911 -992 Carrera 4 GTS Cabrio - N...,Cabrio / Roadster,Unfallfrei,480.0,None,Schwarz,178700,4800.0,10/2023,1.0,DE,4.800 km !!-WIE NEU!!\n1.HAND DEUTSCHES FAHRZE...,https://suchen.mobile.de/fahrzeuge/details.htm...,15,50,2024-03-09 22:09:26.038992+00:00


#### Filter for GT3 or GT 3

In [12]:
df_clean_2 = df_clean_1.loc[
    (df_clean_1["titel"].str.lower().str.contains("gt3|gt 3")) | (df["fahrzeugbescheibung"].str.lower().str.contains("gt3|gt 3"))
].reset_index(drop=True)
df_clean_2

,marke,modell,variante,titel,form,fahrzeugzustand,leistung,getriebe,farbe,preis,kilometer,erstzulassung,fahrzeughalter,standort,fahrzeugbescheibung,url_to_crawl,page_rank,total_num_pages,crawled_timestamp
0,Porsche,992,None,Porsche 992 GT3 SportChrono*Vollschalensitze,Sportwagen / Coupé,Unfallfrei,510.0,Automatik,Grau,181951,104800.0,07/2021,NaN,DE,Sonderausstattung:\nAblage-Paket\nBremssättel ...,https://suchen.mobile.de/fahrzeuge/details.htm...,22,25,2024-03-09 22:09:26.038992+00:00
1,Porsche,992,None,Porsche 992 GT3 CLUBSPORT PTS ACID GREEN - VAT,Sportwagen / Coupé,Unfallfrei,510.0,Automatik,Grün,220680,35600.0,09/2021,NaN,FR,PORSCHE 992 GT3 4.0 510 CLUBSPORT PTS ACID GRE...,https://suchen.mobile.de/fahrzeuge/details.htm...,14,25,2024-03-09 22:09:26.038992+00:00
2,Porsche,992,None,Porsche 992 GT3 Clubsport-Paket Vollschalensitze,Sportwagen / Coupé,Unfallfrei,510.0,Automatik,Rot,199911,21587.0,07/2021,1.0,DE,1. Hand\nunfallfrei\ndeutsches Fahrzeug\nneue...,https://suchen.mobile.de/fahrzeuge/details.htm...,20,25,2024-03-09 22:09:26.038992+00:00
3,Porsche,992,None,Porsche 992 GT3 Touring PDK PCCB PTS Blackoliv...,Sportwagen / Coupé,Unfallfrei,510.0,Automatik,Grün,246800,22900.0,06/2022,2.0,DE,Deutsches Fahrzeug / Unfallfrei / keine Nachl...,https://suchen.mobile.de/fahrzeuge/details.htm...,21,25,2024-03-09 22:09:26.038992+00:00
4,Porsche,992,None,Porsche 992 GT3 Touring PDK PCCB PTS Blackoliv...,Sportwagen / Coupé,Unfallfrei,510.0,Automatik,Grün,246800,22900.0,06/2022,2.0,DE,Deutsches Fahrzeug / Unfallfrei / keine Nachl...,https://suchen.mobile.de/fahrzeuge/details.htm...,1,25,2024-03-09 22:09:26.038992+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340,Porsche,992,None,Porsche 992 911 GT3 TOURING/CARBO/PDK/ 18-WEGE...,Sportwagen / Coupé,Unfallfrei,510.0,Automatik,Grau,209992,4600.0,09/2022,1.0,DE,"Weitere Fotos, Details und Top Angebote finden...",https://suchen.mobile.de/fahrzeuge/details.htm...,6,50,2024-03-09 22:09:26.038992+00:00
341,Porsche,992,None,Porsche 992 911 GT3 Touring LIFT CARBON BOSE C...,Sportwagen / Coupé,Unfallfrei,510.0,Automatik,Schwarz Metallic,229850,429.0,07/2022,1.0,DE,"Herzlich Willkommen bei Elbe Sportwagen, Ihrem...",https://suchen.mobile.de/fahrzeuge/details.htm...,9,50,2024-03-09 22:09:26.038992+00:00
342,Porsche,992,None,Porsche 992 911/992 GT3 Touring*18 Wege*BOSE*S...,Sportwagen / Coupé,Unfallfrei,510.0,Automatik,Grau Metallic,204890,4900.0,04/2022,2.0,DE,Achatgrau metallic\nInnenausst. m. erw. Leder-...,https://suchen.mobile.de/fahrzeuge/details.htm...,2,50,2024-03-09 22:09:26.038992+00:00
343,Porsche,992,None,Porsche 992 GT3 mit Touring Paket Liftsystem B...,Sportwagen / Coupé,Unfallfrei,510.0,Automatik,Silber Metallic,206960,2559.0,10/2021,1.0,DE,Sonderausstattung\nAußen-/Innenspiegel mit Abb...,https://suchen.mobile.de/fahrzeuge/details.htm...,3,50,2024-03-09 22:09:26.038992+00:00


#### Filter for DE

In [13]:
df_clean_3 = df_clean_2[
    (df_clean_2["standort"].str.lower().str.contains("de"))
].reset_index(drop=True)
df_clean_3

,marke,modell,variante,titel,form,fahrzeugzustand,leistung,getriebe,farbe,preis,kilometer,erstzulassung,fahrzeughalter,standort,fahrzeugbescheibung,url_to_crawl,page_rank,total_num_pages,crawled_timestamp
0,Porsche,992,None,Porsche 992 GT3 SportChrono*Vollschalensitze,Sportwagen / Coupé,Unfallfrei,510.0,Automatik,Grau,181951,104800.0,07/2021,NaN,DE,Sonderausstattung:\nAblage-Paket\nBremssättel ...,https://suchen.mobile.de/fahrzeuge/details.htm...,22,25,2024-03-09 22:09:26.038992+00:00
1,Porsche,992,None,Porsche 992 GT3 Clubsport-Paket Vollschalensitze,Sportwagen / Coupé,Unfallfrei,510.0,Automatik,Rot,199911,21587.0,07/2021,1.0,DE,1. Hand\nunfallfrei\ndeutsches Fahrzeug\nneue...,https://suchen.mobile.de/fahrzeuge/details.htm...,20,25,2024-03-09 22:09:26.038992+00:00
2,Porsche,992,None,Porsche 992 GT3 Touring PDK PCCB PTS Blackoliv...,Sportwagen / Coupé,Unfallfrei,510.0,Automatik,Grün,246800,22900.0,06/2022,2.0,DE,Deutsches Fahrzeug / Unfallfrei / keine Nachl...,https://suchen.mobile.de/fahrzeuge/details.htm...,21,25,2024-03-09 22:09:26.038992+00:00
3,Porsche,992,None,Porsche 992 GT3 Touring PDK PCCB PTS Blackoliv...,Sportwagen / Coupé,Unfallfrei,510.0,Automatik,Grün,246800,22900.0,06/2022,2.0,DE,Deutsches Fahrzeug / Unfallfrei / keine Nachl...,https://suchen.mobile.de/fahrzeuge/details.htm...,1,25,2024-03-09 22:09:26.038992+00:00
4,Porsche,992,None,Porsche 992 GT3*APPROVED*CHRONO*LIFT*ALCANTARA...,Sportwagen / Coupé,None,510.0,Automatik,Silber Metallic,183900,39257.0,06/2021,NaN,DE,RS092\nFinanzierung + Leasing ist möglich\nBli...,https://suchen.mobile.de/fahrzeuge/details.htm...,10,25,2024-03-09 22:09:26.038992+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278,Porsche,992,None,Porsche 992 911 GT3 TOURING/CARBO/PDK/ 18-WEGE...,Sportwagen / Coupé,Unfallfrei,510.0,Automatik,Grau,209992,4600.0,09/2022,1.0,DE,"Weitere Fotos, Details und Top Angebote finden...",https://suchen.mobile.de/fahrzeuge/details.htm...,6,50,2024-03-09 22:09:26.038992+00:00
279,Porsche,992,None,Porsche 992 911 GT3 Touring LIFT CARBON BOSE C...,Sportwagen / Coupé,Unfallfrei,510.0,Automatik,Schwarz Metallic,229850,429.0,07/2022,1.0,DE,"Herzlich Willkommen bei Elbe Sportwagen, Ihrem...",https://suchen.mobile.de/fahrzeuge/details.htm...,9,50,2024-03-09 22:09:26.038992+00:00
280,Porsche,992,None,Porsche 992 911/992 GT3 Touring*18 Wege*BOSE*S...,Sportwagen / Coupé,Unfallfrei,510.0,Automatik,Grau Metallic,204890,4900.0,04/2022,2.0,DE,Achatgrau metallic\nInnenausst. m. erw. Leder-...,https://suchen.mobile.de/fahrzeuge/details.htm...,2,50,2024-03-09 22:09:26.038992+00:00
281,Porsche,992,None,Porsche 992 GT3 mit Touring Paket Liftsystem B...,Sportwagen / Coupé,Unfallfrei,510.0,Automatik,Silber Metallic,206960,2559.0,10/2021,1.0,DE,Sonderausstattung\nAußen-/Innenspiegel mit Abb...,https://suchen.mobile.de/fahrzeuge/details.htm...,3,50,2024-03-09 22:09:26.038992+00:00


#### Amend the `form` column

In [14]:
# Spalte E = form = Wenn Sportwagen/Coupe, oder Sportwagen/Coupe, Jahreswagen, oder Sportwagen/Coupe, Neufahrzeug, oder Sportwagen/Coupe, Tageszulassung, oder
# Sportwagen/Coupe, Vorführfahrzeug abgebildet wird, dann auf „Coupé“ ändern!

df_clean_4 = df_clean_3.copy()

# Write a function that checks each value in the column and returns "True" if the value contains "sportwagen/coupé" and "False" otherwise.
def amend_form_col_porsche_992_gt3(x):
    if x.find("sportwagen/coup") != -1:
        return "Coupé"
    else:
        return x
    
# Remove the spaces from the form column and convert to lower case
df_clean_4["form_mod"] = df_clean_4["form"].apply(lambda x: x.replace(" ", "").lower())

# Apply the function amend_form_col on form using the apply method
df_clean_4["form"] = df_clean_4["form_mod"].apply(amend_form_col_porsche_992_gt3)

# Drop the form_mod column
df_clean_4 = df_clean_4.drop("form_mod", axis=1)

# Display the data frame
df_clean_4

,marke,modell,variante,titel,form,fahrzeugzustand,leistung,getriebe,farbe,preis,kilometer,erstzulassung,fahrzeughalter,standort,fahrzeugbescheibung,url_to_crawl,page_rank,total_num_pages,crawled_timestamp
0,Porsche,992,None,Porsche 992 GT3 SportChrono*Vollschalensitze,Coupé,Unfallfrei,510.0,Automatik,Grau,181951,104800.0,07/2021,NaN,DE,Sonderausstattung:\nAblage-Paket\nBremssättel ...,https://suchen.mobile.de/fahrzeuge/details.htm...,22,25,2024-03-09 22:09:26.038992+00:00
1,Porsche,992,None,Porsche 992 GT3 Clubsport-Paket Vollschalensitze,Coupé,Unfallfrei,510.0,Automatik,Rot,199911,21587.0,07/2021,1.0,DE,1. Hand\nunfallfrei\ndeutsches Fahrzeug\nneue...,https://suchen.mobile.de/fahrzeuge/details.htm...,20,25,2024-03-09 22:09:26.038992+00:00
2,Porsche,992,None,Porsche 992 GT3 Touring PDK PCCB PTS Blackoliv...,Coupé,Unfallfrei,510.0,Automatik,Grün,246800,22900.0,06/2022,2.0,DE,Deutsches Fahrzeug / Unfallfrei / keine Nachl...,https://suchen.mobile.de/fahrzeuge/details.htm...,21,25,2024-03-09 22:09:26.038992+00:00
3,Porsche,992,None,Porsche 992 GT3 Touring PDK PCCB PTS Blackoliv...,Coupé,Unfallfrei,510.0,Automatik,Grün,246800,22900.0,06/2022,2.0,DE,Deutsches Fahrzeug / Unfallfrei / keine Nachl...,https://suchen.mobile.de/fahrzeuge/details.htm...,1,25,2024-03-09 22:09:26.038992+00:00
4,Porsche,992,None,Porsche 992 GT3*APPROVED*CHRONO*LIFT*ALCANTARA...,Coupé,None,510.0,Automatik,Silber Metallic,183900,39257.0,06/2021,NaN,DE,RS092\nFinanzierung + Leasing ist möglich\nBli...,https://suchen.mobile.de/fahrzeuge/details.htm...,10,25,2024-03-09 22:09:26.038992+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278,Porsche,992,None,Porsche 992 911 GT3 TOURING/CARBO/PDK/ 18-WEGE...,Coupé,Unfallfrei,510.0,Automatik,Grau,209992,4600.0,09/2022,1.0,DE,"Weitere Fotos, Details und Top Angebote finden...",https://suchen.mobile.de/fahrzeuge/details.htm...,6,50,2024-03-09 22:09:26.038992+00:00
279,Porsche,992,None,Porsche 992 911 GT3 Touring LIFT CARBON BOSE C...,Coupé,Unfallfrei,510.0,Automatik,Schwarz Metallic,229850,429.0,07/2022,1.0,DE,"Herzlich Willkommen bei Elbe Sportwagen, Ihrem...",https://suchen.mobile.de/fahrzeuge/details.htm...,9,50,2024-03-09 22:09:26.038992+00:00
280,Porsche,992,None,Porsche 992 911/992 GT3 Touring*18 Wege*BOSE*S...,Coupé,Unfallfrei,510.0,Automatik,Grau Metallic,204890,4900.0,04/2022,2.0,DE,Achatgrau metallic\nInnenausst. m. erw. Leder-...,https://suchen.mobile.de/fahrzeuge/details.htm...,2,50,2024-03-09 22:09:26.038992+00:00
281,Porsche,992,None,Porsche 992 GT3 mit Touring Paket Liftsystem B...,Coupé,Unfallfrei,510.0,Automatik,Silber Metallic,206960,2559.0,10/2021,1.0,DE,Sonderausstattung\nAußen-/Innenspiegel mit Abb...,https://suchen.mobile.de/fahrzeuge/details.htm...,3,50,2024-03-09 22:09:26.038992+00:00


#### Amend the `fahrzeugzustand` column

In [15]:
# Spalte F = fahrzeugzustand = Wenn Unfallfrei, Nicht fahrtauglich, oder (Leere) abgebildet wird, dann auf “Unfallfrei“ ändern
df_clean_5 = df_clean_4.copy()
def amend_fahrzeugzustand_col_porsche_992_gt3(x):
    if x == "" or x is None:
        return "Unfallfrei"
    elif x.lower().find("unfallfrei") != -1:
        return "Unfallfrei"
    else:
        return x
df_clean_5["fahrzeugzustand"] = df_clean_5["fahrzeugzustand"].apply(amend_fahrzeugzustand_col_porsche_992_gt3)
df_clean_5

,marke,modell,variante,titel,form,fahrzeugzustand,leistung,getriebe,farbe,preis,kilometer,erstzulassung,fahrzeughalter,standort,fahrzeugbescheibung,url_to_crawl,page_rank,total_num_pages,crawled_timestamp
0,Porsche,992,None,Porsche 992 GT3 SportChrono*Vollschalensitze,Coupé,Unfallfrei,510.0,Automatik,Grau,181951,104800.0,07/2021,NaN,DE,Sonderausstattung:\nAblage-Paket\nBremssättel ...,https://suchen.mobile.de/fahrzeuge/details.htm...,22,25,2024-03-09 22:09:26.038992+00:00
1,Porsche,992,None,Porsche 992 GT3 Clubsport-Paket Vollschalensitze,Coupé,Unfallfrei,510.0,Automatik,Rot,199911,21587.0,07/2021,1.0,DE,1. Hand\nunfallfrei\ndeutsches Fahrzeug\nneue...,https://suchen.mobile.de/fahrzeuge/details.htm...,20,25,2024-03-09 22:09:26.038992+00:00
2,Porsche,992,None,Porsche 992 GT3 Touring PDK PCCB PTS Blackoliv...,Coupé,Unfallfrei,510.0,Automatik,Grün,246800,22900.0,06/2022,2.0,DE,Deutsches Fahrzeug / Unfallfrei / keine Nachl...,https://suchen.mobile.de/fahrzeuge/details.htm...,21,25,2024-03-09 22:09:26.038992+00:00
3,Porsche,992,None,Porsche 992 GT3 Touring PDK PCCB PTS Blackoliv...,Coupé,Unfallfrei,510.0,Automatik,Grün,246800,22900.0,06/2022,2.0,DE,Deutsches Fahrzeug / Unfallfrei / keine Nachl...,https://suchen.mobile.de/fahrzeuge/details.htm...,1,25,2024-03-09 22:09:26.038992+00:00
4,Porsche,992,None,Porsche 992 GT3*APPROVED*CHRONO*LIFT*ALCANTARA...,Coupé,Unfallfrei,510.0,Automatik,Silber Metallic,183900,39257.0,06/2021,NaN,DE,RS092\nFinanzierung + Leasing ist möglich\nBli...,https://suchen.mobile.de/fahrzeuge/details.htm...,10,25,2024-03-09 22:09:26.038992+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278,Porsche,992,None,Porsche 992 911 GT3 TOURING/CARBO/PDK/ 18-WEGE...,Coupé,Unfallfrei,510.0,Automatik,Grau,209992,4600.0,09/2022,1.0,DE,"Weitere Fotos, Details und Top Angebote finden...",https://suchen.mobile.de/fahrzeuge/details.htm...,6,50,2024-03-09 22:09:26.038992+00:00
279,Porsche,992,None,Porsche 992 911 GT3 Touring LIFT CARBON BOSE C...,Coupé,Unfallfrei,510.0,Automatik,Schwarz Metallic,229850,429.0,07/2022,1.0,DE,"Herzlich Willkommen bei Elbe Sportwagen, Ihrem...",https://suchen.mobile.de/fahrzeuge/details.htm...,9,50,2024-03-09 22:09:26.038992+00:00
280,Porsche,992,None,Porsche 992 911/992 GT3 Touring*18 Wege*BOSE*S...,Coupé,Unfallfrei,510.0,Automatik,Grau Metallic,204890,4900.0,04/2022,2.0,DE,Achatgrau metallic\nInnenausst. m. erw. Leder-...,https://suchen.mobile.de/fahrzeuge/details.htm...,2,50,2024-03-09 22:09:26.038992+00:00
281,Porsche,992,None,Porsche 992 GT3 mit Touring Paket Liftsystem B...,Coupé,Unfallfrei,510.0,Automatik,Silber Metallic,206960,2559.0,10/2021,1.0,DE,Sonderausstattung\nAußen-/Innenspiegel mit Abb...,https://suchen.mobile.de/fahrzeuge/details.htm...,3,50,2024-03-09 22:09:26.038992+00:00


#### Amend the `Kilometer` column

In [16]:
# Spalte K = kilometer = Wenn (Leere) abgebildet wird, dann auf „1“ ändern. (Info: Sind oftmals Neuwagen und haben Werkskilometer)
df_clean_6 = df_clean_5.copy()
def amend_kilometer_col_porsche_992_gt3(x):
    if x == "" or x is None:
        return 1
    else:
        return x
df_clean_6["kilometer"] = df_clean_6["kilometer"].apply(amend_kilometer_col_porsche_992_gt3)
df_clean_6

,marke,modell,variante,titel,form,fahrzeugzustand,leistung,getriebe,farbe,preis,kilometer,erstzulassung,fahrzeughalter,standort,fahrzeugbescheibung,url_to_crawl,page_rank,total_num_pages,crawled_timestamp
0,Porsche,992,None,Porsche 992 GT3 SportChrono*Vollschalensitze,Coupé,Unfallfrei,510.0,Automatik,Grau,181951,104800.0,07/2021,NaN,DE,Sonderausstattung:\nAblage-Paket\nBremssättel ...,https://suchen.mobile.de/fahrzeuge/details.htm...,22,25,2024-03-09 22:09:26.038992+00:00
1,Porsche,992,None,Porsche 992 GT3 Clubsport-Paket Vollschalensitze,Coupé,Unfallfrei,510.0,Automatik,Rot,199911,21587.0,07/2021,1.0,DE,1. Hand\nunfallfrei\ndeutsches Fahrzeug\nneue...,https://suchen.mobile.de/fahrzeuge/details.htm...,20,25,2024-03-09 22:09:26.038992+00:00
2,Porsche,992,None,Porsche 992 GT3 Touring PDK PCCB PTS Blackoliv...,Coupé,Unfallfrei,510.0,Automatik,Grün,246800,22900.0,06/2022,2.0,DE,Deutsches Fahrzeug / Unfallfrei / keine Nachl...,https://suchen.mobile.de/fahrzeuge/details.htm...,21,25,2024-03-09 22:09:26.038992+00:00
3,Porsche,992,None,Porsche 992 GT3 Touring PDK PCCB PTS Blackoliv...,Coupé,Unfallfrei,510.0,Automatik,Grün,246800,22900.0,06/2022,2.0,DE,Deutsches Fahrzeug / Unfallfrei / keine Nachl...,https://suchen.mobile.de/fahrzeuge/details.htm...,1,25,2024-03-09 22:09:26.038992+00:00
4,Porsche,992,None,Porsche 992 GT3*APPROVED*CHRONO*LIFT*ALCANTARA...,Coupé,Unfallfrei,510.0,Automatik,Silber Metallic,183900,39257.0,06/2021,NaN,DE,RS092\nFinanzierung + Leasing ist möglich\nBli...,https://suchen.mobile.de/fahrzeuge/details.htm...,10,25,2024-03-09 22:09:26.038992+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278,Porsche,992,None,Porsche 992 911 GT3 TOURING/CARBO/PDK/ 18-WEGE...,Coupé,Unfallfrei,510.0,Automatik,Grau,209992,4600.0,09/2022,1.0,DE,"Weitere Fotos, Details und Top Angebote finden...",https://suchen.mobile.de/fahrzeuge/details.htm...,6,50,2024-03-09 22:09:26.038992+00:00
279,Porsche,992,None,Porsche 992 911 GT3 Touring LIFT CARBON BOSE C...,Coupé,Unfallfrei,510.0,Automatik,Schwarz Metallic,229850,429.0,07/2022,1.0,DE,"Herzlich Willkommen bei Elbe Sportwagen, Ihrem...",https://suchen.mobile.de/fahrzeuge/details.htm...,9,50,2024-03-09 22:09:26.038992+00:00
280,Porsche,992,None,Porsche 992 911/992 GT3 Touring*18 Wege*BOSE*S...,Coupé,Unfallfrei,510.0,Automatik,Grau Metallic,204890,4900.0,04/2022,2.0,DE,Achatgrau metallic\nInnenausst. m. erw. Leder-...,https://suchen.mobile.de/fahrzeuge/details.htm...,2,50,2024-03-09 22:09:26.038992+00:00
281,Porsche,992,None,Porsche 992 GT3 mit Touring Paket Liftsystem B...,Coupé,Unfallfrei,510.0,Automatik,Silber Metallic,206960,2559.0,10/2021,1.0,DE,Sonderausstattung\nAußen-/Innenspiegel mit Abb...,https://suchen.mobile.de/fahrzeuge/details.htm...,3,50,2024-03-09 22:09:26.038992+00:00


#### Amend the `Getriebe` col

In [17]:
# Spalte H = getriebe = Wenn getriebe Automatik, dann ändere auf „7-Gang Doppelkupplungsgetriebe (PDK)“
# Spalte H = getriebe = Wenn getriebe Schaltgetriebe, dann ändere auf „6-Gang-GT-Sportschaltgetriebe“.
df_clean_7 = df_clean_6.copy()
def amend_getriebe_col_stg_1_porsche_992_gt3(x):
    if x == "" or x is None:
        return x
    elif x.lower().find("automatik") != -1:
        return "7-Gang Doppelkupplungsgetriebe (PDK)"
    elif x.lower().find("schaltgetriebe") != -1:
        return "6-Gang-GT-Sportschaltgetriebe"
    else:
        return x
    
df_clean_7["getriebe"] = df_clean_7["getriebe"].apply(amend_getriebe_col_stg_1_porsche_992_gt3)
df_clean_7

,marke,modell,variante,titel,form,fahrzeugzustand,leistung,getriebe,farbe,preis,kilometer,erstzulassung,fahrzeughalter,standort,fahrzeugbescheibung,url_to_crawl,page_rank,total_num_pages,crawled_timestamp
0,Porsche,992,None,Porsche 992 GT3 SportChrono*Vollschalensitze,Coupé,Unfallfrei,510.0,7-Gang Doppelkupplungsgetriebe (PDK),Grau,181951,104800.0,07/2021,NaN,DE,Sonderausstattung:\nAblage-Paket\nBremssättel ...,https://suchen.mobile.de/fahrzeuge/details.htm...,22,25,2024-03-09 22:09:26.038992+00:00
1,Porsche,992,None,Porsche 992 GT3 Clubsport-Paket Vollschalensitze,Coupé,Unfallfrei,510.0,7-Gang Doppelkupplungsgetriebe (PDK),Rot,199911,21587.0,07/2021,1.0,DE,1. Hand\nunfallfrei\ndeutsches Fahrzeug\nneue...,https://suchen.mobile.de/fahrzeuge/details.htm...,20,25,2024-03-09 22:09:26.038992+00:00
2,Porsche,992,None,Porsche 992 GT3 Touring PDK PCCB PTS Blackoliv...,Coupé,Unfallfrei,510.0,7-Gang Doppelkupplungsgetriebe (PDK),Grün,246800,22900.0,06/2022,2.0,DE,Deutsches Fahrzeug / Unfallfrei / keine Nachl...,https://suchen.mobile.de/fahrzeuge/details.htm...,21,25,2024-03-09 22:09:26.038992+00:00
3,Porsche,992,None,Porsche 992 GT3 Touring PDK PCCB PTS Blackoliv...,Coupé,Unfallfrei,510.0,7-Gang Doppelkupplungsgetriebe (PDK),Grün,246800,22900.0,06/2022,2.0,DE,Deutsches Fahrzeug / Unfallfrei / keine Nachl...,https://suchen.mobile.de/fahrzeuge/details.htm...,1,25,2024-03-09 22:09:26.038992+00:00
4,Porsche,992,None,Porsche 992 GT3*APPROVED*CHRONO*LIFT*ALCANTARA...,Coupé,Unfallfrei,510.0,7-Gang Doppelkupplungsgetriebe (PDK),Silber Metallic,183900,39257.0,06/2021,NaN,DE,RS092\nFinanzierung + Leasing ist möglich\nBli...,https://suchen.mobile.de/fahrzeuge/details.htm...,10,25,2024-03-09 22:09:26.038992+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278,Porsche,992,None,Porsche 992 911 GT3 TOURING/CARBO/PDK/ 18-WEGE...,Coupé,Unfallfrei,510.0,7-Gang Doppelkupplungsgetriebe (PDK),Grau,209992,4600.0,09/2022,1.0,DE,"Weitere Fotos, Details und Top Angebote finden...",https://suchen.mobile.de/fahrzeuge/details.htm...,6,50,2024-03-09 22:09:26.038992+00:00
279,Porsche,992,None,Porsche 992 911 GT3 Touring LIFT CARBON BOSE C...,Coupé,Unfallfrei,510.0,7-Gang Doppelkupplungsgetriebe (PDK),Schwarz Metallic,229850,429.0,07/2022,1.0,DE,"Herzlich Willkommen bei Elbe Sportwagen, Ihrem...",https://suchen.mobile.de/fahrzeuge/details.htm...,9,50,2024-03-09 22:09:26.038992+00:00
280,Porsche,992,None,Porsche 992 911/992 GT3 Touring*18 Wege*BOSE*S...,Coupé,Unfallfrei,510.0,7-Gang Doppelkupplungsgetriebe (PDK),Grau Metallic,204890,4900.0,04/2022,2.0,DE,Achatgrau metallic\nInnenausst. m. erw. Leder-...,https://suchen.mobile.de/fahrzeuge/details.htm...,2,50,2024-03-09 22:09:26.038992+00:00
281,Porsche,992,None,Porsche 992 GT3 mit Touring Paket Liftsystem B...,Coupé,Unfallfrei,510.0,7-Gang Doppelkupplungsgetriebe (PDK),Silber Metallic,206960,2559.0,10/2021,1.0,DE,Sonderausstattung\nAußen-/Innenspiegel mit Abb...,https://suchen.mobile.de/fahrzeuge/details.htm...,3,50,2024-03-09 22:09:26.038992+00:00


#### Amend the `Marke` col based on the `titel` col

In [42]:
# Spalte D -> A = titel = Wenn im titel „Techart“ steht, dann ändere marke (Spalte A) auf “Porsche/Techart“
# Spalte D -> A = titel = Wenn im titel „Manthey“ steht, dann ändere marke (Spalte A) auf “Porsche/Manthey Performance“
# Spalte D -> A = titel = Wenn im titel „Cup“ steht, dann ändere marke (Spalte A) auf “Porsche/Racing“
df_clean_8 = df_clean_7.copy()
def amend_marke_col_porsche_992_gt3(x, y, replacement_word):
    if x.lower().find(y.lower()) != -1:
        return "Porsche | " + replacement_word

df_clean_8["marke"] = df_clean_8.apply(lambda x: amend_marke_col_porsche_992_gt3(x["titel"], "Techart", "Techart") if x["titel"].lower().find("techart") != -1 else x["marke"], axis=1)
df_clean_8["marke"] = df_clean_8.apply(lambda x: amend_marke_col_porsche_992_gt3(x["titel"], "Manthey", "Manthey") if x["titel"].lower().find("manthey") != -1 else x["marke"], axis=1)
df_clean_8["marke"] = df_clean_8.apply(lambda x: amend_marke_col_porsche_992_gt3(x["titel"], "Cup", "Racing") if x["titel"].lower().find("cup") != -1 else x["marke"], axis=1)
df_clean_8

,marke,modell,variante,titel,form,fahrzeugzustand,leistung,getriebe,farbe,preis,kilometer,erstzulassung,fahrzeughalter,standort,fahrzeugbescheibung,url_to_crawl,page_rank,total_num_pages,crawled_timestamp
0,Porsche,992,None,Porsche 992 GT3 SportChrono*Vollschalensitze,Coupé,Unfallfrei,510.0,7-Gang Doppelkupplungsgetriebe (PDK),Grau,181951,104800.0,07/2021,NaN,DE,Sonderausstattung:\nAblage-Paket\nBremssättel ...,https://suchen.mobile.de/fahrzeuge/details.htm...,22,25,2024-03-09 22:09:26.038992+00:00
1,Porsche,992,None,Porsche 992 GT3 Clubsport-Paket Vollschalensitze,Coupé,Unfallfrei,510.0,7-Gang Doppelkupplungsgetriebe (PDK),Rot,199911,21587.0,07/2021,1.0,DE,1. Hand\nunfallfrei\ndeutsches Fahrzeug\nneue...,https://suchen.mobile.de/fahrzeuge/details.htm...,20,25,2024-03-09 22:09:26.038992+00:00
2,Porsche,992,None,Porsche 992 GT3 Touring PDK PCCB PTS Blackoliv...,Coupé,Unfallfrei,510.0,7-Gang Doppelkupplungsgetriebe (PDK),Grün,246800,22900.0,06/2022,2.0,DE,Deutsches Fahrzeug / Unfallfrei / keine Nachl...,https://suchen.mobile.de/fahrzeuge/details.htm...,21,25,2024-03-09 22:09:26.038992+00:00
3,Porsche,992,None,Porsche 992 GT3 Touring PDK PCCB PTS Blackoliv...,Coupé,Unfallfrei,510.0,7-Gang Doppelkupplungsgetriebe (PDK),Grün,246800,22900.0,06/2022,2.0,DE,Deutsches Fahrzeug / Unfallfrei / keine Nachl...,https://suchen.mobile.de/fahrzeuge/details.htm...,1,25,2024-03-09 22:09:26.038992+00:00
4,Porsche,992,None,Porsche 992 GT3*APPROVED*CHRONO*LIFT*ALCANTARA...,Coupé,Unfallfrei,510.0,7-Gang Doppelkupplungsgetriebe (PDK),Silber Metallic,183900,39257.0,06/2021,NaN,DE,RS092\nFinanzierung + Leasing ist möglich\nBli...,https://suchen.mobile.de/fahrzeuge/details.htm...,10,25,2024-03-09 22:09:26.038992+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278,Porsche,992,None,Porsche 992 911 GT3 TOURING/CARBO/PDK/ 18-WEGE...,Coupé,Unfallfrei,510.0,7-Gang Doppelkupplungsgetriebe (PDK),Grau,209992,4600.0,09/2022,1.0,DE,"Weitere Fotos, Details und Top Angebote finden...",https://suchen.mobile.de/fahrzeuge/details.htm...,6,50,2024-03-09 22:09:26.038992+00:00
279,Porsche,992,None,Porsche 992 911 GT3 Touring LIFT CARBON BOSE C...,Coupé,Unfallfrei,510.0,7-Gang Doppelkupplungsgetriebe (PDK),Schwarz Metallic,229850,429.0,07/2022,1.0,DE,"Herzlich Willkommen bei Elbe Sportwagen, Ihrem...",https://suchen.mobile.de/fahrzeuge/details.htm...,9,50,2024-03-09 22:09:26.038992+00:00
280,Porsche,992,None,Porsche 992 911/992 GT3 Touring*18 Wege*BOSE*S...,Coupé,Unfallfrei,510.0,7-Gang Doppelkupplungsgetriebe (PDK),Grau Metallic,204890,4900.0,04/2022,2.0,DE,Achatgrau metallic\nInnenausst. m. erw. Leder-...,https://suchen.mobile.de/fahrzeuge/details.htm...,2,50,2024-03-09 22:09:26.038992+00:00
281,Porsche,992,None,Porsche 992 GT3 mit Touring Paket Liftsystem B...,Coupé,Unfallfrei,510.0,7-Gang Doppelkupplungsgetriebe (PDK),Silber Metallic,206960,2559.0,10/2021,1.0,DE,Sonderausstattung\nAußen-/Innenspiegel mit Abb...,https://suchen.mobile.de/fahrzeuge/details.htm...,3,50,2024-03-09 22:09:26.038992+00:00


#### Amend the variante column

In [44]:
# Rule 1: Spate A|B|D|G|O verändern Spalte C = variante = Wenn marke Porsche, und modell 992, und leistung 510, und entweder im titel Touring oder in fahrzeugbeschreibung Touring
# steht, dann ändere in Spalte C auf “GT3 mit Touring-Paket“

# Rule 2: Spate A|B|D|G|O verändern Spalte C = variante = Wenn marke Porsche, und Modell 992, und leistung 510, und Spalte C nicht GT3 mit Touring-Paket steht, dann ändere auf “GT3“.

df_clean_9 = df_clean_8.copy()
def amend_variante_col_porsche_992_gt3(x):
    if x["marke"] == "Porsche" and x["modell"] == "992" and x["leistung"] == 510 and (x["titel"].lower().find("touring") != -1 or x["fahrzeugbescheibung"].lower().find("touring") != -1):
        return "GT3 mit Touring-Paket"
    elif x["marke"] == "Porsche" and x["modell"] == "992" and x["leistung"] == 510 and (x["titel"].lower().find("touring") == -1 and x["fahrzeugbescheibung"].lower().find("touring") == -1):
        return "GT3"
    else:
        return x["variante"]

# Rule 1: Spate H = getriebe = Wenn marke Porsche, und Modell 992, und leistung 510, und Variante GT3 oder GT3 mit Touring-Paket, und entweder im titel PDK oder im fahrzeugbeschreibung PDK steht, und getriebe auf (Leere), dann ändere auf “7-Gang Doppelkupplungsgetriebe (PDK)“

# Rule 2: Spate H = getriebe = Wenn marke Porsche, und Modell 992, und leistung 510, und Variante GT3 oder GT3 mit Touring-Paket, und entweder im titel 6-Gang oder im fahrzeugbeschreibung 6-Gang steht, und getriebe auf (Leere), dann ändere auf "6-Gang-GT-Sportschaltgetriebe"
def amend_getriebe_col_stg_2_porsche_992_gt3(x):
    if x["fahrzeugbescheibung"] is None:
        return x["getriebe"]
    elif x["marke"] == "Porsche" and x["modell"] == "992" and x["leistung"] == 510\
    and (x["variante"] == "GT3" or x["variante"] == "GT3 mit Touring-Paket")\
    and (x["titel"].lower().find("pdk") != -1 or x["fahrzeugbescheibung"].lower().find("pdk") != -1) and (x["getriebe"] == "" or x["getriebe"] is None):
        return "7-Gang Doppelkupplungsgetriebe (PDK)"
    elif x["marke"] == "Porsche" and x["modell"] == "992" and x["leistung"] == 510\
    and (x["variante"] == "GT3" or x["variante"] == "GT3 mit Touring-Paket")\
    and (x["titel"].lower().find("6-gang") != -1 or x["fahrzeugbescheibung"].lower().find("6-gang") != -1) and (x["getriebe"] == "" or x["getriebe"] is None):
        return "6-Gang-GT-Sportschaltgetriebe"
    else:
        return x["getriebe"]

# Rule 3: Spate H = getriebe = Wenn marke Porsche, und Modell 992, und leistung 510, und Variante GT3 oder GT3 mit Touring-Paket, und getriebe auf (Leere), dann ändere auf “7-Gang Doppelkupplungsgetriebe (PDK)“.“.
def amend_getriebe_col_stg_3_porsche_992_gt3(x):
    if x["marke"] == "Porsche" and x["modell"] == "992" and x["leistung"] == 510\
    and (x["variante"] == "GT3" or x["variante"] == "GT3 mit Touring-Paket")\
    and (x["getriebe"] == "" or x["getriebe"] is None):
        return "7-Gang Doppelkupplungsgetriebe (PDK)"
    else:
        return x["getriebe"]
    
# Apply the functions
df_clean_9["variante"] = df_clean_9.apply(
    lambda x: amend_variante_col_porsche_992_gt3(x), axis=1
)

# Getriebe Stg 2
df_clean_9["getriebe"] = df_clean_9.apply(lambda x: amend_getriebe_col_stg_2_porsche_992_gt3(x), axis=1)

# Getriebe Stg 3
df_clean_9["getriebe"] = df_clean_9.apply(lambda x: amend_getriebe_col_stg_3_porsche_992_gt3(x), axis=1)

In [45]:
display(df_clean_9[df_clean_9["getriebe"].isnull()])

display(df_clean_9["getriebe"].value_counts())

,marke,modell,variante,titel,form,fahrzeugzustand,leistung,getriebe,farbe,preis,kilometer,erstzulassung,fahrzeughalter,standort,fahrzeugbescheibung,url_to_crawl,page_rank,total_num_pages,crawled_timestamp
141,Porsche,992,None,"Porsche 992 GT3 RS, Weissach Paket, Lift, PCCB",Coupé,Unfallfrei,525.0,None,Gelb,395000,37.0,06/2023,1.0,DE,Zum Verkauf steht ein Porsche 992 GT3 RS Neuwa...,https://suchen.mobile.de/fahrzeuge/details.htm...,18,50,2024-03-09 22:09:26.038992+00:00
209,Porsche,992,None,"Porsche 992 GT3 RS CLUBSPORTPAKET,LIFTING,SCHA...",Coupé,Unfallfrei,525.0,None,Grau,359992,625.0,03/2023,NaN,DE,Traumhafter Porsche 992 GT3 RS Clubsportpaket...,https://suchen.mobile.de/fahrzeuge/details.htm...,14,50,2024-03-09 22:09:26.038992+00:00


getriebe
7-Gang Doppelkupplungsgetriebe (PDK)    254
6-Gang-GT-Sportschaltgetriebe            27
Name: count, dtype: int64

#### New column `ausstattung`

In [47]:
# Neue Spalte D = Wenn marke Porsche, und Modell 992, und Variante GT3 oder GT3 mit Touring-Paket, und form Coupe, und fahrzeugzustand unfallfrei, und Leistung 510, und im titel PTS steht, dann ändere auf „PTS“.
# Neue Spalte D = Wenn marke Porsche, und Modell 992, und Variante GT3 oder GT3 mit Touring-Paket, und form Coupe, und fahrzeugzustand unfallfrei, und Leistung 510, und entweder im titel PCCB oder im Fahrzeugbeschreibung PCCB steht, dann ändere auf „PCCB“. (Info: Wenn das modell schon die Ausstattung PTS hat und auch PCCB, dann ändere auf „PTS | PCCB“)
# Neue Spalte D = Wenn marke Porsche, und Modell 992, und Variante GT3 oder GT3 mit Touring-Paket, und form Coupe, und fahrzeugzustand unfallfrei, und Leistung 510, und entweder im titel Manufa oder im Fahrzeugbeschreibung Manufa steht, dann ändere auf „Exklusive Manufaktur“.

df_clean_10 = df_clean_9.copy()
def amend_exklusive_manufaktur_col_porsche_992_gt3(x):
    if x["fahrzeugbescheibung"] is None:
        return None
    elif x["marke"] == "Porsche" and x["modell"] == "992" and (x["variante"] == "GT3" or x["variante"] == "GT3 mit Touring-Paket")\
    and x["form"] == "Coupé" and x["fahrzeugzustand"].lower() == "unfallfrei" and x["leistung"] == 510\
    and (
        ("pts" in x["titel"].lower() and "pccb" in x["titel"].lower()) or\
        ("pts" in x["fahrzeugbescheibung"].lower() and "pccb" in x["fahrzeugbescheibung"].lower())
    ):
        return "PTS + PCCB"
    elif x["marke"] == "Porsche" and x["modell"] == "992" and (x["variante"] == "GT3" or x["variante"] == "GT3 mit Touring-Paket")\
    and x["form"] == "Coupé" and x["fahrzeugzustand"].lower() == "unfallfrei" and x["leistung"] == 510\
    and (x["titel"].lower().find("pts") != -1):
        return "PTS"
    elif x["marke"] == "Porsche" and x["modell"] == "992" and (x["variante"] == "GT3" or x["variante"] == "GT3 mit Touring-Paket")\
    and x["form"] == "Coupé" and x["fahrzeugzustand"] == "Unfallfrei" and x["leistung"] == 510\
    and (x["titel"].lower().find("pccb") != -1 or x["fahrzeugbescheibung"].lower().find("pccb") != -1):
        return "PCCB"
    else:
        return None

# Apply the function
df_clean_10["ausstattung"] = df_clean_10.apply(lambda x: amend_exklusive_manufaktur_col_porsche_992_gt3(x), axis=1)

# Move the Austattung column to be between "variante" and "titel"
austattung_col = df_clean_10.pop("ausstattung")
df_clean_10.insert(3, "ausstattung", austattung_col)

df_clean_10

,marke,modell,variante,ausstattung,titel,form,fahrzeugzustand,leistung,getriebe,farbe,preis,kilometer,erstzulassung,fahrzeughalter,standort,fahrzeugbescheibung,url_to_crawl,page_rank,total_num_pages,crawled_timestamp
0,Porsche,992,GT3,None,Porsche 992 GT3 SportChrono*Vollschalensitze,Coupé,Unfallfrei,510.0,7-Gang Doppelkupplungsgetriebe (PDK),Grau,181951,104800.0,07/2021,NaN,DE,Sonderausstattung:\nAblage-Paket\nBremssättel ...,https://suchen.mobile.de/fahrzeuge/details.htm...,22,25,2024-03-09 22:09:26.038992+00:00
1,Porsche,992,GT3,None,Porsche 992 GT3 Clubsport-Paket Vollschalensitze,Coupé,Unfallfrei,510.0,7-Gang Doppelkupplungsgetriebe (PDK),Rot,199911,21587.0,07/2021,1.0,DE,1. Hand\nunfallfrei\ndeutsches Fahrzeug\nneue...,https://suchen.mobile.de/fahrzeuge/details.htm...,20,25,2024-03-09 22:09:26.038992+00:00
2,Porsche,992,GT3 mit Touring-Paket,PTS + PCCB,Porsche 992 GT3 Touring PDK PCCB PTS Blackoliv...,Coupé,Unfallfrei,510.0,7-Gang Doppelkupplungsgetriebe (PDK),Grün,246800,22900.0,06/2022,2.0,DE,Deutsches Fahrzeug / Unfallfrei / keine Nachl...,https://suchen.mobile.de/fahrzeuge/details.htm...,21,25,2024-03-09 22:09:26.038992+00:00
3,Porsche,992,GT3 mit Touring-Paket,PTS + PCCB,Porsche 992 GT3 Touring PDK PCCB PTS Blackoliv...,Coupé,Unfallfrei,510.0,7-Gang Doppelkupplungsgetriebe (PDK),Grün,246800,22900.0,06/2022,2.0,DE,Deutsches Fahrzeug / Unfallfrei / keine Nachl...,https://suchen.mobile.de/fahrzeuge/details.htm...,1,25,2024-03-09 22:09:26.038992+00:00
4,Porsche,992,GT3,None,Porsche 992 GT3*APPROVED*CHRONO*LIFT*ALCANTARA...,Coupé,Unfallfrei,510.0,7-Gang Doppelkupplungsgetriebe (PDK),Silber Metallic,183900,39257.0,06/2021,NaN,DE,RS092\nFinanzierung + Leasing ist möglich\nBli...,https://suchen.mobile.de/fahrzeuge/details.htm...,10,25,2024-03-09 22:09:26.038992+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278,Porsche,992,GT3 mit Touring-Paket,None,Porsche 992 911 GT3 TOURING/CARBO/PDK/ 18-WEGE...,Coupé,Unfallfrei,510.0,7-Gang Doppelkupplungsgetriebe (PDK),Grau,209992,4600.0,09/2022,1.0,DE,"Weitere Fotos, Details und Top Angebote finden...",https://suchen.mobile.de/fahrzeuge/details.htm...,6,50,2024-03-09 22:09:26.038992+00:00
279,Porsche,992,GT3 mit Touring-Paket,None,Porsche 992 911 GT3 Touring LIFT CARBON BOSE C...,Coupé,Unfallfrei,510.0,7-Gang Doppelkupplungsgetriebe (PDK),Schwarz Metallic,229850,429.0,07/2022,1.0,DE,"Herzlich Willkommen bei Elbe Sportwagen, Ihrem...",https://suchen.mobile.de/fahrzeuge/details.htm...,9,50,2024-03-09 22:09:26.038992+00:00
280,Porsche,992,GT3 mit Touring-Paket,None,Porsche 992 911/992 GT3 Touring*18 Wege*BOSE*S...,Coupé,Unfallfrei,510.0,7-Gang Doppelkupplungsgetriebe (PDK),Grau Metallic,204890,4900.0,04/2022,2.0,DE,Achatgrau metallic\nInnenausst. m. erw. Leder-...,https://suchen.mobile.de/fahrzeuge/details.htm...,2,50,2024-03-09 22:09:26.038992+00:00
281,Porsche,992,GT3 mit Touring-Paket,None,Porsche 992 GT3 mit Touring Paket Liftsystem B...,Coupé,Unfallfrei,510.0,7-Gang Doppelkupplungsgetriebe (PDK),Silber Metallic,206960,2559.0,10/2021,1.0,DE,Sonderausstattung\nAußen-/Innenspiegel mit Abb...,https://suchen.mobile.de/fahrzeuge/details.htm...,3,50,2024-03-09 22:09:26.038992+00:00


In [48]:
# Save the data frame to a CSV file
df_clean_10.to_csv("porsche_992_gt3.csv", index=False)